# Notes

- improve using these
    - [Finetuning Efficientnet](https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/vision/ipynb/image_classification_efficientnet_fine_tuning.ipynb#scrollTo=1KvkMvLI0iCh)
    - [Wandb Article](https://wandb.ai/stacey/mendeleev/reports/Tables-Tutorial-Visualize-Data-for-Image-Classification--VmlldzozNjE3NjA)
    - [bloodMNIST](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/keras/Keras_pipeline_with_Weights_and_Biases.ipynb#scrollTo=-303Shsata7Z)

    - [Simpsons classification](https://colab.research.google.com/drive/181GCGp36_75C2zm7WLxr9U2QjMXXoibt#scrollTo=xftEwKyuaJ5q)

## Efficientnet Finetuning Notes

- too low- [0.000001, 0.000005, 0.00005]
- too high [0.002, 0.005]
- 0.001, 0.0005 works
    - why?
    - isn't it too high for finetuning?

## Efficientnet to do
- should lr be min max or a list [0.01, 0.005, 0.001, 0.0005, 0.0001]
- freeze parameter - [true. false]
    - so 2 best models, one with freeze true and another with freeze false
    - [freeze: true best model](https://wandb.ai/rock-classifiers/Whats-this-rockv2/runs/9fgg9eua?workspace=user-udaylunawat)
        
    val_accuracy - 63.9

    val_f1_score - 63.4
            earlystopping_patience 3
            freeze true
            lr 0.00007
            lr_reduce_factor 0.9
            lr_reduce_patience 1
    - [freeze: false best model](https://wandb.ai/rock-classifiers/Whats-this-rockv2/runs/4qmss0xh/overview?workspace=user-udaylunawat)
    
    val_accuracy:-0.74
    
    val_f1_score:-0.73

            lr:- 0.0003
            reduce_factor: 0.3
            lr_reduce_patience:- 3
            earlystopping_patience:- 10

    - finetune both models in another sweep

In [ ]:
# try:
#     import keras_cv
# except:
#     print("Install Tensorflow >= 2.9 and Restart runtime.")

# Clone Repo

In [ ]:
!rm -rf /content/Whats-this-rock/
!git clone https://github.com/udaylunawat/Whats-this-rock.git
%cd /content/Whats-this-rock/

Cloning into 'Whats-this-rock'...
remote: Enumerating objects: 1330, done.
remote: Counting objects: 100% (278/278), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 1330 (delta 180), reused 181 (delta 88), pack-reused 1052
Receiving objects: 100% (1330/1330), 3.27 MiB | 23.91 MiB/s, done.
Resolving deltas: 100% (848/848), done.
/content/Whats-this-rock


In [ ]:
import os

if 'WANDB_API_KEY' not in os.environ:
  from getpass import getpass
  secret = getpass('Enter WandB API Key: ')
  os.environ['WANDB_API_KEY'] = secret

Enter WandB API Key: ··········


# Deploy Telegram Bot

Uploading secrets with telegram key

In [ ]:
import os

from getpass import getpass
telegram_token = getpass('Enter Telegram API Key: ')
os.environ['TOKEN'] = telegram_token

Enter Telegram API Key: ··········


In [ ]:
%%bash
pip install -r requirements.txt --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires keras<2.9,>=2.8.0rc0, but you have keras 2.9.0 which is incompatible.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.9.1 which is incompatible.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorflow-estimator<2.9,>=2.8, but you have tensorflow-estimator 2.9.0 which is incompatible.
google-colab 1.0.0 requires tornado~=5.1.0, but you have tornado 6.2 which is incompatible.


In [ ]:
!python src/bot.py

Downloaded Trained model: rock-classifiers/Whats-this-rockv2/cvzc7hq0.
Model loaded!
Bot started!
Please visit t.me/test7385_bot to start using me!
Telegram Bot Deployed!
Exiting immediately!


In [ ]:
import wandb
api = wandb.Api()

run = api.run("rock-classifiers/Whats-this-rockv2/cvzc7hq0")
run.file('model-best.h5').download()

<_io.TextIOWrapper name='./model-best.h5' mode='r' encoding='UTF-8'>

In [ ]:
for i in run.files():
    print(i)

<File artifact/174765146/wandb_manifest.json (application/json) 899.0B>
<File code/src/train.py (text/x-python; charset=utf-8) 5.7KiB>
<File config.yaml () 1.4KiB>
<File diff.patch (text/plain) 2.4KiB>
<File media/images/Classification Report Image:_49_4e09aa405c52575608c9.png (image/png) 202.5KiB>
<File media/plotly/Confusion Matrix_48_c48bc37a6a931cf7ec4e.plotly.json (application/json) 16.7KiB>
<File model-best.h5 (application/keras) 86.5MiB>
<File output.log () 4.4KiB>
<File requirements.txt (text/plain; charset=utf-8) 7.1KiB>
<File wandb-metadata.json (application/json) 1.3KiB>
<File wandb-summary.json (application/json) 1.0KiB>


In [ ]:
config

Available objects for config:
     AliasManager
     ColabHistoryManager
     DisplayFormatter
     IPCompleter
     IPKernelApp
     InlineBackend
     LoggingMagics
     MagicsManager
     OSMagics
     PrefilterManager
     ScriptMagics
     Shell
     StoreMagics


# Training and evaluating Model

# Setup

In [ ]:
# %%writefile setup.sh
# #!/bin/bash

# # setting up kaggle
# wget https://www.dropbox.com/s/ltp4ly8ilvxlgas/kaggle.json
# mkdir -p ~/.kaggle
# mv kaggle.json ~/.kaggle
# chmod 600 /root/.kaggle/kaggle.json

# pip install -r requirements-dev.txt

# # setting up data dir
# rm -rf data/1_extracted data/2_processed data/3_consume data/4_tfds_dataset data/corrupted_images
# mkdir -p data/0_raw data/1_extracted data/2_processed data/3_consume data/4_tfds_dataset data/corrupted_images checkpoints
# mkdir -p data/2_processed/Basalt/
# mkdir -p data/2_processed/Granite/
# mkdir -p data/2_processed/Marble/
# mkdir -p data/2_processed/Quartzite/
# mkdir -p data/2_processed/Limestone/
# mkdir -p data/2_processed/Sandstone/

# # dataset 1 processing
# kaggle datasets download salmaneunus/rock-classification --path data/0_raw/
# unzip -q data/0_raw/rock-classification.zip -d data/1_extracted/
# mv data/1_extracted/Dataset/Igneous/* data/2_processed
# mv data/1_extracted/Dataset/Metamorphic/* data/2_processed
# mv data/1_extracted/Dataset/Sedimentary/* data/2_processed

# # # dataset 2 processing
# # kaggle datasets download mahmoudalforawi/igneous-metamorphic-sedimentary-rocks-and-minerals --path data/0_raw/
# # unzip -q data/0_raw/igneous-metamorphic-sedimentary-rocks-and-minerals.zip -d data/1_extracted/
# # # https://serverfault.com/a/267266/979238
# # mv --backup=t data/1_extracted/Rock_Dataset/igneous\ rocks/Basalt/* data/2_processed/Basalt/
# # mv --backup=t data/1_extracted/Rock_Dataset/igneous\ rocks/granite/* data/2_processed/Granite/
# # mv --backup=t data/1_extracted/Rock_Dataset/metamorphic\ rocks/marble/* data/2_processed/Marble/
# # mv --backup=t data/1_extracted/Rock_Dataset/metamorphic\ rocks/quartzite/* data/2_processed/Quartzite/
# # mv --backup=t data/1_extracted/Rock_Dataset/sedimentary\ rocks/coal/* data/2_processed/Coal/
# # mv --backup=t data/1_extracted/Rock_Dataset/sedimentary\ rocks/Limestone/* data/2_processed/Limestone/
# # mv --backup=t data/1_extracted/Rock_Dataset/sedimentary\ rocks/Sandstone/* data/2_processed/Sandstone/
# # rm -rf data/1_extracted/Rock_Dataset/minerals

# # # dataset 3 processing
# # wget --quiet -O data/0_raw/dataset3.zip https://github.com/SmartPracticeschool/llSPS-INT-3797-Rock-identification-using-deep-convolution-neural-network/raw/master/dataset.zip
# # unzip -q data/0_raw/dataset3.zip -d data/1_extracted/
# # mv --backup=t data/1_extracted/dataset/*/Basalt/* data/2_processed/Basalt/
# # mv --backup=t data/1_extracted/dataset/*/Granite/* data/2_processed/Granite/
# # mv --backup=t data/1_extracted/dataset/*/Marble/* data/2_processed/Marble/
# # mv --backup=t data/1_extracted/dataset/*/Quartzite/* data/2_processed/Quartzite/
# # mv --backup=t data/1_extracted/dataset/*/Limestone/* data/2_processed/Limestone/
# # mv --backup=t data/1_extracted/dataset/*/Sandstone/* data/2_processed/Sandstone/

# # # dataset 4 processing
# # kaggle datasets download neelgajare/rocks-dataset --path data/0_raw/
# # unzip -q data/0_raw/rocks-dataset.zip -d data/1_extracted/
# # cp -r --backup=t data/1_extracted/Rocks/Basalt/* data/2_processed/Basalt/
# # cp -r --backup=t data/1_extracted/Rocks/Granite/* data/2_processed/Granite/
# # cp -r --backup=t data/1_extracted/Rocks/Marble/* data/2_processed/Marble/
# # cp -r --backup=t data/1_extracted/Rocks/Quartzite/* data/2_processed/Quartzite/
# # cp -r --backup=t data/1_extracted/Rocks/Coal/* data/2_processed/Coal/
# # cp -r --backup=t data/1_extracted/Rocks/Limestone/* data/2_processed/Limestone/
# # cp -r --backup=t data/1_extracted/Rocks/Sandstone/* data/2_processed/Sandstone/


In [ ]:
!nvidia-smi
!sh setup.sh
!python src/preprocess.py

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 18 not upgraded.
Need to get 430 MB of archives.
After this operation, 3,139 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.1.0.77-1+cuda11.2 [430 MB]
Fetched 430 MB in 9s (49.3 MB/s)
(Reading database ... 155676 files and directories currently installed.)
Removing libcudnn8-dev (8.0.5.39-1+cuda11.1) ...
(Reading database ... 155654 

# Load Data and Train

In [ ]:
import json
default = dict(
    notes='''
    
    Datasets used:[1, 2, 3, 4], sampling: None, class-weights:True, 
    preprocessing:rescale=1./255, horizontal_flip=True, vertical_flip=True
    
    ''',
    model_name="inceptionresnetv2",
    augment=True,
    freeze=False,
    finetune=True,
    pretrained_model_link= "rock-classifiers/Whats-this-rockv2/cvzc7hq0",
    optimizer="adam",
    lr=.0005,
    batch_size=32,
    max_epochs=30,
    image_size=224,
    loss_fn='categorical_crossentropy',
    metrics=['accuracy'],
    earlystopping_patience=10,
    earlystopping_min_delta = 0.02,
    lr_reduce_patience=3,
    lr_reduce_factor=.5,
    threshold=.7,
)

# save dictionary to config.json file
with open('config.json', 'w') as f:
    json.dump(default, f)

In [ ]:
default['pretrained_model_link'].split('/')

['rock-classifiers', 'Whats-this-rockv2', 'cvzc7hq0']

# Train

In [ ]:
os.environ['WANDB_MODE'] = 'offline' # offline

In [ ]:
!python src/train.py

In [ ]:
# !python src/evaluate.py

# Weights and Biases Sweeps

In [ ]:
!git pull

Updating 004570f..8240747
Fast-forward
 src/src.data.utils.py | 12 +++++++++---
 src/models.py         |  1 +
 sweep.yaml            | 27 +++++++++++----------------
 3 files changed, 21 insertions(+), 19 deletions(-)


In [ ]:
  lr:- 0.0003
  reduce_factor: 0.3
  lr_reduce_patience:- 3
  earlystopping_patience:- 10

In [ ]:
%%writefile sweep.yaml
method: bayes
metric:
  goal: maximize
  name: val_f1_score
parameters:
  notes:
    value: "Datasets:[1,2,3,4], train shuffle :True, Augment: 9 augmentations, 
            sampling: None, removing_corrupt: False"
  augment:
    values: [true]
  finetune:
    values: [false]
  freeze:
    values: [false]
  batch_size:
    values: [32]
  metrics:
    value: ["accuracy"]
  max_epochs:
    value: 100
  loss_fn:
    value: "categorical_crossentropy"
  lr:
    distribution: uniform
    max: 0.001
    min: 0.00005
  model_name:
    values: ["efficientnet"]
  optimizer:
    values: ["adam"]
  image_size:
    value: 224
  earlystopping_patience:
    value: 10
  earlystopping_min_delta:
    value: 0.02
  lr_reduce_patience:
    values: [1, 2, 3, 4]
  lr_reduce_factor:
    values: [.9, .7, .3, .1],
threshold:
    value: 0.7
early_terminate:
  type: hyperband
  s: 2
  eta: 3
  max_iter: 100
program: src/train.py

Writing sweep.yaml


In [ ]:
!wandb sweep \
    --project Whats-this-rockv2 \
    --entity rock-classifiers \
    sweep.yaml

wandb: Creating sweep from: sweep.yaml
wandb: Created sweep with ID: 81jd5gu7
wandb: View sweep at: https://wandb.ai/rock-classifiers/Whats-this-rockv2/sweeps/81jd5gu7
wandb: Run sweep agent with: wandb agent rock-classifiers/Whats-this-rockv2/81jd5gu7


In [ ]:
!wandb agent rock-classifiers/Whats-this-rockv2/f9rl3654